## Introdução

Nosso projeto consiste em avaliar o processo de visto americano para trabalho, o H1B, e se dependendo de certos fatores uma pessa receberia o visto ou não. Para tirar este visto é necessário passar informações como, o nome do lugar de trabalho, com o que a pessoa trabalhará, se é um trabalho de tempo integral, o salário, e a cidade onde será o trabalho (traduzido para latitude e longitude), todos dados contidos no nosso dataset. Porém havia um problema nosso dataset, obtido no Kaggle havia 3 milhões de linhas, um arquivo muito grande, fazendo com que fique difícil e demorado para se trabalhar. Por esse motivo cortamos o dataset, no qual o processo esta em um outro notebook chamado Shuffle dataset.

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.discrete.discrete_model as sm
from sklearn.model_selection import train_test_split
from sklearn import svm
import pickle

In [3]:
df = pd.read_csv('Dataframe2.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,0,2692228,0,"FLORIOGRAPHY, LLC",Market Research Analysts,MARKET RESEARCH ANALYST,Y,40456.0,2011.0,"KATY, TEXAS",-95.824396,29.785785
1,1,1772788,1,MINDTREE LIMITED,"Computer Occupations, All Other",TEST MANAGER,Y,63357.0,2014.0,"FRANKLIN, TENNESSEE",-86.868890,35.925064
2,2,2488118,1,INFOSYS LIMITED,Computer Systems Analysts,MODULE LEAD,Y,39166.0,2012.0,"ROANOKE, VIRGINIA",-79.941427,37.270970
3,3,1463659,1,TATA CONSULTANCY SERVICES LIMITED,Computer Programmers,COMPUTER PROGRAMMER,Y,63690.0,2014.0,"SAN DIEGO, CALIFORNIA",-117.161084,32.715738
4,4,1448818,1,"AMERICAN IT RESOURCE GROUP, INC","Software Developers, Applications",SOFTWARE ENGINEER,Y,62712.0,2014.0,"SCHAUMBURG, ILLINOIS",-88.083406,42.033361


Na célula abaixo, o tratamento dos dados é feito. Em primeiro lugar as colunas Unnamed: 0 e Unnamed: 0.1, são retiradas, essas correspondem ao index atual e ao index antes do embaralhamento feito no outro notebook. O local de trabalho, WORKSITE, também é retirado uma vez que esse já está no dataset em forma de Latitude e Longitude, variáves quantitavas. Essas então são transformadas para valores de 0 a 1 para facilitar a forma de se trabalhar, bem como a variável de trabalho em tempo intergral, FULL_TIME_POSITION, também é transformada, de N ou Y, para 0 ou 1. JOB_TITLE é retirado pois a variável SOC_NAME, do Inglês Standard Occupational Classification (SOC), já corresponde ao trabalho ocupado porém em uma forma de classificação padrão e essa então é transormada para uma váriavel quantitativa através do metodo de One-hot Encoding. Por fim o ano é retirado uma vez que queremos prever o visto de anos futuros então não se pode levar em conta os anos passados.  

In [4]:
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def mapp (value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


df = df.drop(["WORKSITE","YEAR","JOB_TITLE","Unnamed: 0","Unnamed: 0.1"], axis=1)
df["SOC_NAME"] = df["SOC_NAME"].apply(lambda x: x.upper())

df = df.drop(["EMPLOYER_NAME"], axis=1)

df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].apply(lambda x: 0 if x=="N" else 1)



latmin = df["lat"].min()
latmax = df["lat"].max()
lonmin = df["lon"].min()
lonmax = df["lon"].max()

df["lat"] = df["lat"].apply(lambda x: mapp(x, latmin, latmax, 0, 1))
df["lon"] = df["lon"].apply(lambda x: mapp(x, lonmin, lonmax, 0, 1))

df = dummify(df, "SOC_NAME")

In [5]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,SOC_NAME_ACTUARIES,SOC_NAME_ADHESIVE BONDING MACHINE OPERATORS AND TENDERS,...,"SOC_NAME_TREASURERS, CONTROLLERS, AND CHIEF FINANCIAL OFFIC",SOC_NAME_URBAN AND REGIONAL PLANNERS,SOC_NAME_VETERINARIANS,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, POSTSECONDARY",SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,0,1,40456.0,0.204336,0.318062,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,63357.0,0.233835,0.437502,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,39166.0,0.256653,0.463687,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,63690.0,0.134054,0.375065,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,62712.0,0.229834,0.556339,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O X e y do modelo, baseando-se no CASE_STATUS(se foi aprovado ou não) são separados em base para treinamentos e base para teste, sendo a divisão feita de 67 e 33 por cento. Em seguida é aplicado o modelo, neste caso de regressão logística, uma vez que esse método consegue fazer a predição de para valores a partir das nossas variáves que foram transformadas em quantitativas.

In [7]:
X = df.drop(["CASE_STATUS"], axis=1).values
y = df["CASE_STATUS"].values

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=RANDOM_SEED)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6700, 470), (6700,), (3300, 470), (3300,))

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Acurácia: {}".format(100*accuracy_score(y_test, y_pred)))

Acurácia: 64.78787878787878


In [10]:
filename = 'finalized_model.p'
pickle.dump(model, open(filename, 'wb'))

In [12]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6478787878787878


In [10]:
df.head()

In [13]:
import statsmodels.discrete.discrete_model as sm

model2 = sm.Logit(y_train,X_train)
model2.fit()

C:\Users\Freddy Dratwa\AppData\Local\Programs\Python\Python37\lib\site-packages\statsmodels\discrete\discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


         Current function value: inf
         Iterations: 35


C:\Users\Freddy Dratwa\AppData\Local\Programs\Python\Python37\lib\site-packages\statsmodels\discrete\discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [15]:
df["CASE_STATUS"][df["CASE_STATUS"]==1].size/(df["CASE_STATUS"][df["CASE_STATUS"]==1].size+df["CASE_STATUS"][df["CASE_STATUS"]==0].size)

0.5

## Conclusão

A partir dos resultados obtidos, percebe-se que o modelo tem uma precisão de quase 97%. Desta forma, conclui-se que o modelo é bastante preciso. Portanto, se uma pessoa deseja saber se terá seu visto para os EUA concedido, basta que ele use suas informações no modelo. a resposta obtida terá 96% de probabilidade de ser igual à da vida real.